In [17]:
import os 
os.getcwd()
import pickle
data = pickle.load(open('/data/jh/notebooks/mengluyao/filter_1_lists.pkl','rb'))

In [18]:
import jieba
import jieba.analyse
jieba.analyse.set_stop_words("/data/share/xueyou/stop_words.txt")

In [19]:
data[0][0]

{'article_type': '谍照',
 'author': '腾讯汽车',
 'brand': '["\\u82f1\\u83f2\\u5c3c\\u8fea"]',
 'content': '近日，从官方获悉东风英菲尼迪全新QX50已经于2018年4月26日正式下线。新车搭载拥有可变压缩比技术的2.0T发动机，并将推出6款车型，预售35万元起，将于6月正式上市。 <para>外观方面，国产全新QX50前脸采用了英菲尼迪家族标志性的波澜中网，视觉效果具有较高的辨识度。全新的前大灯组造型犀利，内部采用全LED光源。此外，该车的前保险杠造型也更加动感。 <para>车尾部分，新车尾部采用多条横向弧线轮廓，整体造型饱满，一条弧形镀铬饰条与两侧全新尾灯组相连接，双边共双出镀铬排气为该车增添了运动气息。车身尺寸方面，新车的长宽高分别为4705×1903×1678mm，轴距为2798mm。 <para>内饰方面，新车中控采用偏向驾驶员一侧的设计风格，中控台配备两个竖向排列的多媒体屏幕，科技感十足。仪表盘采用双机械指针仪表+中央液晶屏幕设计，整体造型简约规整。 <para>动力方面，新车搭载两种不同调校的2.0T发动机，最大功率为180/200kW，峰值扭矩均为380Nm。该发动机型配有可变压缩比、双循环，以及双喷射（缸内直喷+歧管喷射）技术，其中压缩比可在8:1到14:1之间自由调节。传动方面，与之匹配的是一台XTRONIC CVT无级变速箱。',
 'headline': '东风英菲尼迪全新QX50部分配置曝光 预售35万起',
 'pubdate': '2018-04-28T06:43:00Z\n',
 'serial': '["QX50"]'}

In [20]:
file_stop = open("/data/share/xueyou/stop_words.txt")
stop_words = []
for line in file_stop.readlines():
    if line.strip():
        stop_words.append(line[:-1])
        
file_stop.close()

In [21]:
stop_words.append(['para',' '])

In [22]:
#过滤停用词函数
def filter_stop_words(wlist,stop_words):
    res = []
    for word in wlist:
        if word in stop_words:
            continue
        res.append(word)
    return res

#whether_similar函数
def similar_news(art1,art2,stop_words):
    art1_words = filter_stop_words(jieba.lcut(art1['content'], HMM = True),stop_words)
    art2_words = filter_stop_words(jieba.lcut(art2['content'], HMM = True),stop_words)
    if min(len(set(art1_words)), len(set(art2_words))) > 0:
        a = len(set(art1_words)&set(art2_words))/min(len(set(art1_words)), len(set(art2_words)))
        if a > 0.8:
            return True
        else:
            return False
    return False

In [26]:
#对每一个list中相同来源author的dict的具有相似的'content'的新闻过滤
def author_list(articles_list):
    res = []
    for art in articles_list:
        res.append(art['author'])
    return list(set(res))

def same_author_news(author,news):
    res = []
    for article in news:
        if article['author'] == author:
            res.append(article)
    return res
        
def filter_list_news(list_news,stop_words):
    res = []
    for author in author_list(list_news):    #提取新闻list中的作者种类，对每一类分别处理
        temp_list = []
        the_news = same_author_news(author, list_news)
        for news in the_news:
            if not temp_list:
                temp_list.append(news)
            else:
                similar_tag = False
                for article0 in temp_list:   #进行比较
                    if similar_news(news, article0, stop_words):
                        similar_tag = True
                        break
                if not similar_tag:
                    temp_list.append(news)              
        res.extend(temp_list)
                        
    return res

In [27]:
filter_data = []
for news_list in data:
    filter_data.append(filter_list_news(news_list,stop_words))

In [28]:
print(len(filter_data))
nums = [len(a) for a in filter_data]
print(sum(nums))


20821
38272


In [31]:
import pickle
file = open('/data/share/mengluyao/filter_lists_jieba.pkl','wb')
pickle.dump(filter_data,file)
file.close()

In [37]:
filter_data[1300]

[{'article_type': '新车发布',
  'author': '太平洋汽车',
  'brand': '["\\u5b9d\\u9a6c", "MINI", "\\u5954\\u9a70", "\\u4e30\\u7530", "\\u5b9d\\u9a6c", "\\u798f\\u7279", "\\u5954\\u9a70", "JEEP", "\\u5927\\u4f17", "\\u672c\\u7530", "\\u798f\\u7279", "\\u5965\\u8fea", "\\u798f\\u7279", "\\u5b9d\\u9a6c", "\\u5927\\u4f17", "\\u672c\\u7530", "\\u672c\\u7530", "\\u8bb4\\u6b4c", "\\u4e30\\u7530", "\\u4e30\\u7530", "MINI", "\\u5927\\u4f17", "\\u5e7f\\u6c7d\\u4f20\\u797a", "\\u5b9d\\u9a6c"]',
  'content': '1 新一代奔驰G级/AMG CLS53/ AMG E53 回顶部 <para>作为每年年初的首场国际大车展，北美车展总能带给我们新年的第一波惊喜，美国本土厂商必定不遗余力发布新车，欧洲、日韩厂家也不会怠慢美国市场，诸如新一代奔驰G级、新一代Jeep牧马人这种硬货系数登场；坚持颜值即正义的朋友则能看看新一代奥迪A7；要论话题性，看看新一代 丰田Avalon 的前脸，你就知道为什么那么多人对这台车又爱又恨。 新一代奔驰G级 <para>新车看点：老硬汉焕发新春 <para>新一代 奔驰G级 在2017北美车展上全球首发亮相，新车依旧沿用了经典的“方盒子”车身造型。据悉，该车预计将在2018年年中登陆中国。 <para>作为一款拥有近40年历史的经典车型，G级从诞生之初，方盒子的形状就基本没变过。此次换代，整体造型自然也不会发生太大调整，不然新老车主都不会认可。 <para>外观上能将新一代G级和现款区分的部位在大灯组、格栅和外后视镜。如果觉得外观没什么新意，不妨进车内看看，工程师和设计师的功力可都花在了里面。车内可选碳纤维、金属、黑钢琴和木质饰板，另有丰富配色可供选择。 <para>新一代 奔驰 G级的内

In [14]:
print(similar_news(filter_data[1][3],filter_data[1][4],stop_words))

True


In [2]:
data[1]

NameError: name 'data' is not defined

In [4]:
data[1]

[{'article_type': '新车发布',
  'author': '腾讯汽车',
  'brand': '["\\u5b9d\\u9a6c"]',
  'content': '在2018北京车展上，宝马8系概念车正式亮相。新车定位为大型双门GT跑车，并采用了全新的设计语言，其量产版——全新8系Coupe将于今年6月15日发布。 <para>宝马8系概念车采用了一套全新的设计语言，进气格栅由曾经的分体演变成为整体，并且尺寸更大造型也更加立体，竖条幅格栅也进化成了类似字母Y的形状。大灯采用了更加简洁的造型，并使用激光大灯和LED光源勾勒出极具未来感的“天使眼”灯组。 <para>车身侧面，经典的霍夫迈斯特拐角得以延续，但在角度上更加用心，从而不破坏大型双门GT跑车优美舒展的侧窗造型。相比之下新车所匹配的21英寸双五辐轮圈和金色刹车卡钳，则更加突出新车的运动性。尾部翘起的扰流板设计，更突出了8系概念车应有的运动天赋，后包围的空气动力学设计很好的与前包围呼应，并采用双边共两出的矩形排气设计。 <para>相比于外观设计语言的进化，8系概念车的内饰设计相比现行的宝马风格可谓发生了翻天覆地的变化。一体式中控台采用碳纤维和皮革相搭配，并镶嵌有大尺寸液晶显示屏，整体来看非常简约。 <para>此外，新车还配备有全液晶仪表盘、极具未来感的方向盘已经全新的电子换挡杆设计。作为一款大型双门GT跑车，新车采用2+2式的座椅布局。动力方面，宝马并没有公开8系概念车搭载的具体动力信息，但是相关负责人透露新车未来量产后的入门级版本将从直列6缸发动机起步。作为宝马未来的旗舰车型，我们有足够的理由相信新车能够搭载V8或V12发动机。',
  'headline': '宝马全新8系概念车亮相 定位大型双门GT跑车',
  'pubdate': '2018-04-25T05:44:00Z\n',
  'serial': '["8\\u7cfb"]'},
 {'article_type': '新车发布',
  'author': '凤凰汽车',
  'brand': '["\\u5b9d\\u9a6c"]',
  'content': '日前， 宝马 官方发布了一张全新8系量产版车型的预告图。图片近景主体是已经发布的M8 GT E赛车，旁边则是一辆蓝色的全新8系量产版车型。有外媒报道称6月15日

In [112]:
len(data[0][0]['content'])

529

In [113]:
jieba.lcut(data[0][0]['content'], HMM = True)

['近日',
 '，',
 '从',
 '官方',
 '获悉',
 '东风',
 '英菲尼迪',
 '全新',
 'QX50',
 '已经',
 '于',
 '2018',
 '年',
 '4',
 '月',
 '26',
 '日',
 '正式',
 '下线',
 '。',
 '新车',
 '搭载',
 '拥有',
 '可变',
 '压缩比',
 '技术',
 '的',
 '2.0',
 'T',
 '发动机',
 '，',
 '并',
 '将',
 '推出',
 '6',
 '款',
 '车型',
 '，',
 '预售',
 '35',
 '万元',
 '起',
 '，',
 '将',
 '于',
 '6',
 '月',
 '正式',
 '上市',
 '。',
 ' ',
 '<',
 'para',
 '>',
 '外观',
 '方面',
 '，',
 '国产',
 '全新',
 'QX50',
 '前脸',
 '采用',
 '了',
 '英菲尼迪',
 '家族',
 '标志性',
 '的',
 '波澜',
 '中',
 '网',
 '，',
 '视觉效果',
 '具有',
 '较',
 '高',
 '的',
 '辨识',
 '度',
 '。',
 '全新',
 '的',
 '前大灯',
 '组',
 '造型',
 '犀利',
 '，',
 '内部',
 '采用',
 '全',
 'LED',
 '光源',
 '。',
 '此外',
 '，',
 '该车',
 '的',
 '前',
 '保险杠',
 '造型',
 '也',
 '更加',
 '动感',
 '。',
 ' ',
 '<',
 'para',
 '>',
 '车尾',
 '部分',
 '，',
 '新车',
 '尾部',
 '采用',
 '多条',
 '横向',
 '弧线',
 '轮廓',
 '，',
 '整体',
 '造型',
 '饱满',
 '，',
 '一条',
 '弧形',
 '镀铬',
 '饰条',
 '与',
 '两侧',
 '全新',
 '尾灯',
 '组',
 '相连接',
 '，',
 '双边',
 '共双出',
 '镀铬',
 '排气',
 '为',
 '该车',
 '增添',
 '了',
 '运动',
 '气息',
 '。',
 '车身',
 '尺寸',
 '方面',
 '，',

In [114]:
print(filter_stop_words(jieba.lcut(data[0][0]['content'], HMM = True),stop_words))

['近日', '官方', '获悉', '东风', '英菲尼迪', '全新', 'QX50', '已经', '2018', '年', '4', '月', '26', '日', '正式', '下线', '新车', '搭载', '拥有', '可变', '压缩比', '技术', '2.0', 'T', '发动机', '推出', '6', '款', '车型', '预售', '35', '万元', '6', '月', '正式', '上市', ' ', '外观', '方面', '国产', '全新', 'QX50', '前脸', '采用', '英菲尼迪', '家族', '标志性', '波澜', '中', '网', '视觉效果', '具有', '高', '辨识', '度', '全新', '前大灯', '组', '造型', '犀利', '内部', '采用', '全', 'LED', '光源', '该车', '前', '保险杠', '造型', '更加', '动感', ' ', '车尾', '部分', '新车', '尾部', '采用', '多条', '横向', '弧线', '轮廓', '整体', '造型', '饱满', '一条', '弧形', '镀铬', '饰条', '两侧', '全新', '尾灯', '组', '相连接', '双边', '共双出', '镀铬', '排气', '该车', '增添', '运动', '气息', '车身', '尺寸', '方面', '新车', '长宽', '高', '分别', '4705', '1903', '1678mm', '轴距', '2798mm', ' ', '内饰', '方面', '新车', '中控', '采用', '偏向', '驾驶员', '一侧', '设计', '风格', '中控台', '配备', '两个', '竖向', '排列', '多媒体', '屏幕', '科技', '感', '十足', '仪表盘', '采用', '双', '机械', '指针', '仪表', '中央', '液晶屏幕', '设计', '整体', '造型', '简约', '规整', ' ', '动力', '方面', '新车', '搭载', '两种', '不同', '调校', '2.0', 'T', '发动机', '最大', '功率', '180', '200kW', '峰值', '

In [47]:
import jieba
string = data[1][2]['content']
type(jieba.lcut_for_search(string))

list

In [49]:
seg_list = jieba.cut(string, cut_all = True)
print('full model :'+ '/'.join(seg_list))

full model :宝马/8/系/概念/概念车/在/这次/北京/车展/上/正式/亮相///新车/采用/了/夸张/激进/的/外观/外观设计/设计///定位/为/双门/GT/跑车////para//新车/的/量产/版/车型////宝马/8/系/Coupe/也/将/于/6/月/15/日/在/勒芒/赛道/全球/首发///次日/赛车/版/的/M8/GTE/将/参加/加勒/勒芒/24/小时/耐力/耐力赛////para//宝马/8/系/概念/概念车/的/外观/外观设计/设计/比较/夸张///车身/线条/和/标志/标志性/的/双/肾/进气/格/栅/都/能/给/人/一种/动感/的/视觉/视觉效果/效果///但/我们/预计/计量/量产/产后/的/宝马/8/系/在外/外观/和/内饰/设计/上会/更加/加内/内敛///更加/符合/大众/的/审美/审美观/美观////para//据悉/全新/新一代/一代/宝马/8/系/将/采用/与/全新/7/系/相同/的/CLAR/平台/打造///既然/是/CLAR/平台///减重/就/会/被/列入/重要/考量/范围///预计/未来/量产/版/车型/将/采用/大量/轻量/轻量化/量化/的/车身/身材/材料///从而/在/改善/操控/的/同时/带来/更好/的/燃油/经济/经济性////para//据/相关/负责/负责人/责人/透露///未来/的/宝马/8/系/将/提供/直列/六/缸/发动/发动机/动机/作为/入门/动力///此外/新车/还有/可能/提供/V8/V12/和/插/电/式/混合/动力/三种/不同/动力/的/车型//


In [51]:
seg_list = jieba.lcut_for_search(string)
print('search model: '+ '/'.join(seg_list))

search model: 宝马/8/系/概念/概念车/在/这次/北京/车展/上/正式/亮相/，/新车/采用/了/夸张/激进/的/外观/设计/外观设计/，/定位/为/双门/GT/跑车/。/ /</para/>/新车/的/量产/版/车型/—/—/宝马/8/系/Coupe/也/将/于/6/月/15/日/在/勒芒/赛道/全球/首发/，/次日/赛车/版/的/M8/ /GTE/将/参加/勒芒/24/小时/耐力/耐力赛/。/ /</para/>/宝马/8/系/概念/概念车/的/外观/设计/外观设计/比较/夸张/，/车身/线条/和/标志/标志性/的/双/肾/进气/格栅/都/能/给/人/一种/动感/的/视觉/效果/视觉效果/，/但/我们/预计/量产/后/的/宝马/8/系在/外观/和/内饰/设计/上/会/更加/内敛/，/更加/符合/大众/的/审美/美观/审美观/。/ /</para/>/据悉/全新/一代/宝马/8/系将/采用/与/全新/7/系/相同/的/CLAR/平台/打造/，/既然/是/CLAR/平台/，/减重/就/会/被/列入/重要/考量/范围/，/预计/未来/量产/版/车型/将/采用/大量/轻量/量化/轻量化/的/车身/材料/，/从而/在/改善/操控/的/同时/带来/更好/的/燃油/经济/经济性/。/ /</para/>/据/相关/负责/责人/负责人/透露/，/未来/的/宝马/8/系将/提供/直列/六缸/发动/动机/发动机/作为/入门/动力/，/此外/新车/还有/可能/提供/V8/、/V12/和/插/电式/混合/动力/三种/不同/动力/的/车型/。


In [52]:
seg_list = jieba.cut(string)
print('preision model: '+'/'.join(seg_list))

preision model: 宝马/8/系/概念车/在/这次/北京/车展/上/正式/亮相/，/新车/采用/了/夸张/激进/的/外观设计/，/定位/为/双门/GT/跑车/。/ /</para/>/新车/的/量产/版/车型/—/—/宝马/8/系/Coupe/也/将/于/6/月/15/日/在/勒芒/赛道/全球/首发/，/次日/赛车/版/的/M8/ /GTE/将/参加/勒芒/24/小时/耐力赛/。/ /</para/>/宝马/8/系/概念车/的/外观设计/比较/夸张/，/车身/线条/和/标志性/的/双/肾/进气/格栅/都/能/给/人/一种/动感/的/视觉效果/，/但/我们/预计/量产/后/的/宝马/8/系在/外观/和/内饰/设计/上/会/更加/内敛/，/更加/符合/大众/的/审美观/。/ /</para/>/据悉/全新/一代/宝马/8/系将/采用/与/全新/7/系/相同/的/CLAR/平台/打造/，/既然/是/CLAR/平台/，/减重/就/会/被/列入/重要/考量/范围/，/预计/未来/量产/版/车型/将/采用/大量/轻量化/的/车身/材料/，/从而/在/改善/操控/的/同时/带来/更好/的/燃油/经济性/。/ /</para/>/据/相关/负责人/透露/，/未来/的/宝马/8/系将/提供/直列/六缸/发动机/作为/入门/动力/，/此外/新车/还有/可能/提供/V8/、/V12/和/插/电式/混合/动力/三种/不同/动力/的/车型/。


In [53]:
seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造
